# GMO Forecasting

*Case: Grantham, Mayo, and Van Otterloo, 2012: Estimating the Equity Risk Premium
[9-211-051].*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from openpyxl import load_workbook

FILE = '../data/gmo_analysis_data.xlsx'

wb = load_workbook(FILE)
print(wb.sheetnames)

info = pd.read_excel(FILE, sheet_name='info')
signals = pd.read_excel(FILE, sheet_name='signals', parse_dates=True, index_col=0)
rf = pd.read_excel(FILE, sheet_name='risk-free rate', parse_dates=True, index_col=0)
total_rets = pd.read_excel(FILE, sheet_name='total returns', parse_dates=True, index_col=0)
info.head()

['info', 'signals', 'risk-free rate', 'total returns']


,Ticker,Description
0,SPX D/P,S&P 500 Index for dividend price ratio
1,SPX E/P,S&P 500 Index for earning price ratio
2,T-Note 10YR,10-year Treasury yield
3,TBill 3M,3-month T-Bill rate
4,SPY,SPY ETF for returns and dividend yield


In [2]:
total_rets.head()

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.023292,-0.022094,-0.013000
1997-01-31,0.061786,0.014735,0.034448
1997-02-28,0.009565,0.022265,0.012733
1997-03-31,-0.045721,-0.015152,-0.016441
1997-04-30,0.064368,-0.006731,0.000000


In [3]:
rf.head() # rate is annualized

,TBill 3M
date,
1996-12-31,0.05171
1997-01-31,0.05147
1997-02-28,0.05220
1997-03-31,0.05322
1997-04-30,0.05233


## 2 Analyzing GMO

_This section utilizes data in the file `gmo_data.xlsx`._ Convert total returns to **excess returns** using the risk‑free rate.


In [4]:
# Align dataframes to have matching indices and columns before subtraction
rets = total_rets.align(rf, join='inner', axis=0)[0].copy()
for col in total_rets.columns:
    rets[col] = total_rets[col] - rf['TBill 3M'] / 12

# Add the risk-free rate column itself if desired:
rets['TBill 3M'] = rf['TBill 3M'] / 12
total_rets['TBill 3M'] = rf['TBill 3M'] / 12
rets.head()

,SPY,GMWAX,GMGEX,TBill 3M
date,,,,
1996-12-31,-0.027601,-0.026403,-0.017309,0.004309
1997-01-31,0.057497,0.010446,0.030159,0.004289
1997-02-28,0.005215,0.017915,0.008383,0.004350
1997-03-31,-0.050156,-0.019587,-0.020876,0.004435
1997-04-30,0.060008,-0.011092,-0.004361,0.004361


1. **Performance (GMWAX).** Compute **mean**, **volatility**, and **Sharpe ratio** for **GMWAX** over three samples:
   - inception → 2011
   - 2012 → present
   - inception → present  
   Has the mean, vol, and Sharpe changed much since the case?


In [5]:
def maximum_drawdown(returns, relative=False, start_date=None, end_date=None):
    if start_date is not None:
        returns = returns[start_date:]
    if end_date is not None:
        returns = returns[:end_date]
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max

    max_drawdown = drawdown.min()
    end_date = drawdown.idxmin()
    summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})

    if relative:
        max_drawdown = (max_drawdown - returns.mean())/returns.std()

    for col in drawdown:
        # Compute the date of the peak preceding the maximum drawdown
        summary.loc[col, 'Peak'] = rolling_max.loc[:end_date[col], col].idxmax()
        recovery = drawdown.loc[end_date[col]:, col]
        try:
            # Find the first date at which the drawdown fully recovers (>= 0)
            recover_date = recovery[recovery >= 0].index[0]
            summary.loc[col, 'Recover'] = recover_date
        except Exception:
            summary.loc[col, 'Recover'] = pd.NaT

        # Ensure Peak and Recover are both Timestamps, not arrays or Series,
        # and cast to date (drop hours)
        try:
            summary.loc[col, 'Peak'] = pd.to_datetime(summary.loc[col, 'Peak']).date()
        except Exception:
            summary.loc[col, 'Peak'] = pd.NaT
        try:
            summary.loc[col, 'Recover'] = pd.to_datetime(summary.loc[col, 'Recover']).date()
        except Exception:
            summary.loc[col, 'Recover'] = pd.NaT

        # Compute recovery duration if both Peak and Recover are present and valid
        if pd.notnull(summary.loc[col, 'Peak']) and pd.notnull(summary.loc[col, 'Recover']):
            # Convert to pd.Timestamp for subtraction, then timedelta, then just days:
            try:
                duration = (
                    pd.Timestamp(summary.loc[col, 'Recover']) -
                    pd.Timestamp(summary.loc[col, 'Peak'])
                ).days
                summary.loc[col, 'Duration (to Recover)'] = f"{duration} days" if pd.notnull(duration) else pd.NaT
            except Exception:
                summary.loc[col, 'Duration (to Recover)'] = pd.NaT
        else:
            summary.loc[col, 'Duration (to Recover)'] = pd.NaT

        summary = summary[['Max Drawdown','Peak','Bottom','Recover','Duration (to Recover)']]

    return summary    


def performance_metrics(returns, annualization=1, quantile=.05, relative=False, mdd=True, start_date=None, end_date=None):
    metrics = pd.DataFrame(index=returns.columns)

    if start_date is not None:
        returns = returns[start_date:]

    if end_date is not None:
        returns = returns[:end_date]

    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()

    metrics['Skewness'] = returns.skew()
    metrics['Kurtosis'] = returns.kurtosis()

    VaR = returns.quantile(quantile)
    CVaR = (returns[returns < returns.quantile(quantile)]).mean()

    if relative:
        VaR = (VaR - returns.mean())/returns.std()
        CVaR = (CVaR - returns.mean())/returns.std()

    metrics[f'VaR ({quantile})'] = VaR
    metrics[f'CVaR ({quantile})'] = CVaR

    if mdd:
        mdd_stats = maximum_drawdown(returns)
        metrics = metrics.join(mdd_stats)

        if relative:
            metrics['Max Drawdown'] = (metrics['Max Drawdown'] - returns.mean())/returns.std()
    return metrics

incep_sample_metrics = performance_metrics(rets, annualization=12, end_date="2011", mdd=False)
sec_sample_metrics = performance_metrics(rets, annualization=12, start_date="2012", mdd=False)
full_sample_metrics = performance_metrics(rets, annualization=12, mdd=False)
incep_sample_max_drawdown_metrics = maximum_drawdown(total_rets, end_date="2011")
sec_sample_max_drawdown_metrics = maximum_drawdown(total_rets, start_date="2012")
full_sample_max_drawdown_metrics = maximum_drawdown(total_rets)

# build hierarchical index according to sample to display metrics for all samples
# Concatenate metrics with a sample indicator column
metrics_list = [
    ('Inception-2011', incep_sample_metrics, incep_sample_max_drawdown_metrics),
    ('2012-Present', sec_sample_metrics, sec_sample_max_drawdown_metrics),
    ('Inception-Present', full_sample_metrics, full_sample_max_drawdown_metrics)
]
# Concatenate metrics and max drawdown DataFrames side by side for each sample, adding a 'Sample' column to each
metrics_df = pd.concat(
    [
        pd.concat([metrics, max_drawdown], axis=1).assign(Sample=name)
        for name, metrics, max_drawdown in metrics_list
    ],
    axis=0
)
metrics_df = metrics_df.reset_index().set_index(['Sample', 'index'])
metrics_df.index.names = ['Sample', 'Fund']
display(metrics_df)

/var/folders/7x/mk6b_27178v6y3m4bk5dnbkc0000gn/T/ipykernel_37150/3564056289.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2007-10-31' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  summary.loc[col, 'Peak'] = pd.to_datetime(summary.loc[col, 'Peak']).date()
/var/folders/7x/mk6b_27178v6y3m4bk5dnbkc0000gn/T/ipykernel_37150/3564056289.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2010-10-29' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  summary.loc[col, 'Recover'] = pd.to_datetime(summary.loc[col, 'Recover']).date()
/var/folders/7x/mk6b_27178v6y3m4bk5dnbkc0000gn/T/ipykernel_37150/3564056289.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pan

Mean       Vol    Sharpe       Min       Max  \
Sample            Fund                                                         
Inception-2011    SPY       0.035824  0.164163  0.218225 -0.165550  0.109160   
                  GMWAX     0.046422  0.110499  0.420110 -0.149150  0.081877   
                  GMGEX    -0.003823  0.147253 -0.025963 -0.151592  0.096042   
                  TBill 3M  0.028338  0.006038  4.693492 -0.000013  0.005324   
2012-Present      SPY       0.134974  0.139736  0.965917 -0.124693  0.126918   
                  GMWAX     0.049157  0.092661  0.530503 -0.115018  0.074458   
                  GMGEX     0.013182  0.228077  0.057794 -0.658863  0.124668   
                  TBill 3M  0.015678  0.005508  2.846637 -0.000017  0.004552   
Inception-Present SPY       0.083256  0.153416  0.542680 -0.165550  0.126918   
                  GMWAX     0.047730  0.102209  0.466986 -0.149150  0.081877   
                  GMGEX     0.004312  0.189982  0.022695 -0.658863  0.124668   
                  TBill 3M  0.022281  0.006064  3.674409 -0.000017  0.005324   

                            Skewness   Kurtosis  VaR (0.05)  CVaR (0.05)  \
Sample            Fund                                                     
Inception-2011    SPY      -0.514212   0.582770   -0.080228    -0.107769   
                  GMWAX    -0.891709   3.058298   -0.044003    -0.077413   
                  GMGEX    -0.509564   0.672829   -0.082292    -0.100367   
                  TBill 3M  0.006044  -1.573984    0.000038     0.000014   
2012-Present      SPY      -0.504899   0.874237   -0.061134    -0.084464   
                  GMWAX    -0.546601   2.108603   -0.039814    -0.056840   
                  GMGEX    -6.192453  61.007825   -0.065603    -0.151964   
                  TBill 3M  0.956801  -0.623396    0.000004    -0.000002   
Inception-Present SPY      -0.550973   0.776528   -0.078272    -0.098433   
                  GMWAX    -0.778067   2.884218   -0.041147    -0.067310   
                  GMGEX    -5.183655  59.654743   -0.075737    -0.127666   
                  TBill 3M  0.415903  -1.429939    0.000013     0.000002   

                            Max Drawdown        Peak     Bottom     Recover  \
Sample            Fund                                                        
Inception-2011    SPY          -0.507976  2007-10-31 2009-02-27         NaT   
                  GMWAX        -0.293614  2007-10-31 2009-02-27  2010-10-29   
                  GMGEX        -0.555630  2007-10-31 2009-02-27         NaT   
                  TBill 3M     -0.000013  2011-09-30 2011-10-31         NaT   
2012-Present      SPY          -0.239280  2021-12-31 2022-09-30  2023-12-29   
                  GMWAX        -0.216795  2021-05-31 2022-09-30  2024-02-29   
                  GMGEX        -0.737364  2014-06-30 2016-11-30         NaT   
                  TBill 3M     -0.000017  2015-07-31 2015-09-30  2015-10-30   
Inception-Present SPY          -0.507976  2007-10-31 2009-02-27  2012-03-30   
                  GMWAX        -0.293614  2007-10-31 2009-02-27  2010-10-29   
                  GMGEX        -0.761812  2007-10-31 2016-11-30         NaT   
                  TBill 3M     -0.000017  2015-07-31 2015-09-30  2015-10-30   

                           Duration (to Recover)  
Sample            Fund                            
Inception-2011    SPY                        NaT  
                  GMWAX                1094 days  
                  GMGEX                      NaT  
                  TBill 3M                   NaT  
2012-Present      SPY                   728 days  
                  GMWAX                1004 days  
                  GMGEX                      NaT  
                  TBill 3M               91 days  
Inception-Present SPY                  1612 days  
                  GMWAX                1094 days  
                  GMGEX                      NaT  
                  TBill 3M               91 days

Interestingly, the mean, vol and Sharpe of GWMAX have not changed much between subsamples - they've remained around 4.6-4.9\%, 9-11\%, and .4-.54 respectively.


2. **Tail risk (GMWAX).** For all three samples, analyze extreme scenarios:
   - minimum return
   - 5th percentile (VaR‑5th)
   - maximum drawdown (compute on **total** returns, not excess returns)  
   (a) Does GMWAX have high or low tail‑risk as seen by these stats?  
   (b) Does that vary much across the two subsamples?


(a) GMWAX has lower tail risk than SPY as measured by VaR(0.05), min returns, and max drawdowns. In particular, its VaR is considerably lower than SPY's, and its worst drawdown preceding the GFC (coinciding with SPY's) is -29.3\% in comparison with SPY's halving.

(b) The difference in tail risk statistics between the two samples is mostly in the maximum drawdown. While the second sample had the pandemic shock, the first one had the global financial crisis. As a reult, VaR and min returns aren't that different.


3. **Market exposure (GMWAX).** For all three samples, regress **excess returns of GMWAX** on **excess returns of SPY**:
   - report estimated **alpha**, **beta**, and **R²**
   - is GMWAX a **low‑beta** strategy? has that changed since the case?
   - does GMWAX provide **alpha**? has that changed across subsamples?


In [6]:
from warnings import filterwarnings
filterwarnings('ignore')

# regress GMWAX on SPY
# first sample
def regress_y_on_X(rets, y, X: list[str] | str, start_date = None, end_date = None):
    if isinstance(X, str):
        X = [X]
    X = rets[X]
    y = rets[y]
    if start_date is not None:
        X = X.loc[start_date:]
        y = y.loc[start_date:]
    if end_date is not None:
        X = X.loc[:end_date]
        y = y.loc[:end_date]
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    return results


def print_regression_results(title: str, results):
    print("==========================================")
    print(title)
    print("alpha: {:.6f}".format(results.params[0]))
    print("beta: {:.6f}".format(results.params[1]))
    print("R²: {:.6f}".format(results.rsquared))
    print("==========================================")

def get_regression_results_dataframe(results):
    return pd.DataFrame({
        'alpha': results.params[0],
        'beta': results.params[1],
        'R²': results.rsquared
    }, index=[0])

first_sample_results = regress_y_on_X(rets, 'GMWAX', 'SPY', None, '2011')
first_s_res = (get_regression_results_dataframe(first_sample_results))

second_sample_results = regress_y_on_X(rets, 'GMWAX', 'SPY', '2012', None)
second_s_res = (get_regression_results_dataframe(second_sample_results))

full_sample_results = regress_y_on_X(rets, 'GMWAX', 'SPY', None, None)
full_s_res = (get_regression_results_dataframe(full_sample_results))

# label and concatenate the dataframes
all_res = pd.concat([first_s_res, second_s_res, full_s_res])
all_res.index = ['Inception-2011', '2012-Present', 'Inception-Present']
display(all_res)

,alpha,beta,R²
Inception-2011,0.002250,0.542128,0.648686
2012-Present,-0.002280,0.566914,0.730904
Inception-Present,0.000179,0.547452,0.675236


GMWAX is a middling beta strategy, and that hasn't changed. It is a low-alpha strategy, however, and its alpha has switched signs between samples.

4. **Compare to GMGEX.** Repeat items 1–3 for **GMGEX**. What are key differences between the two strategies?

In [7]:
first_gmgex_res =  get_regression_results_dataframe(regress_y_on_X(rets, 'GMGEX', 'SPY', None, '2011'))
second_gmgex_res = get_regression_results_dataframe(regress_y_on_X(rets, 'GMGEX', 'SPY', '2012', None))
full_gmgex_res = get_regression_results_dataframe(regress_y_on_X(rets, 'GMGEX', 'SPY', None, None))

all_gmgex_res = pd.concat([first_gmgex_res, second_gmgex_res, full_gmgex_res])
all_gmgex_res.index = ['Inception-2011', '2012-Present', 'Inception-Present']
display(all_gmgex_res)

,alpha,beta,R²
Inception-2011,-0.002600,0.764237,0.725898
2012-Present,-0.008139,0.821257,0.253171
Inception-Present,-0.005064,0.781633,0.398403


From the first to the second sample, GMGEX has had significant changes in mean returns (negative to slightly positive), volatility (22\% in later sample versus 14\% in the first), keeping a low absolute Sharpe.

Its tail statistics are worse than SPY or GMWAX across the board (except for its minimum monthly return in the first period being slightly less negative than SPY's worst month). The maximum drawdown and worst days for the second sample seem like measurement error - they are awful. They'd imply an extremely high tail-risk strategy. Finally, it's a much higher beta strategy than GMWAX, though it has lower $R^2$ with SPY over the second sample. Its alpha is consistenly negative as opposed to GMWAX.

## 3 Forecast Regressions

_This section utilizes data in `gmo_data.xlsx`._

1. **Lagged regression.** Consider the regression with predictors lagged one period:

$$
r^{SPY}_{t} \;=\; \alpha^{SPY,X} \;+\; \big(\beta^{SPY,X}\big)^\prime X_{t-1} \;+\; \epsilon^{SPY,X}_{t}
\tag{1}
$$

Estimate (1) and report the **$R^2$**, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for:
   - $X$ as a single regressor, the **dividend–price** ratio ($DP$)
   - $X$ as a single regressor, the **earnings–price** ratio ($EP$)
   - $X$ with **three** regressors: $DP$, $EP$, and the **10‑year yield**  
   For each, report the **$R^2$**.


2. **Trading strategy from forecasts.** For each of the three regressions:
   - Build the forecasted SPY return: $\hat r^{SPY}_{t+1}$ (forecast made using $X_t$ to predict $r^{SPY}_{t+1}$).
   - Set the scale (portfolio weight) to $w_t = 100 \,\hat r^{SPY}_{t+1}$.
   - Strategy return: $r^x_{t+1} = w_t\, r^{SPY}_{t+1}$.  
   For each strategy, compute:
   - mean, volatility, Sharpe
   - max drawdown
   - market **alpha**
   - market **beta**
   - market **information ratio**

3. **Risk characteristics.**
   - For both strategies, the market, and GMO, compute monthly **VaR** at $\pi = 0.05$ (use the historical quantile).
   - The case mentions stocks under‑performed short‑term bonds from 2000–2011. Does the dynamic portfolio above under‑perform the risk‑free rate over this time?
   - Based on the regression estimates, in how many periods do we estimate a **negative risk premium**?
   - Do you believe the dynamic strategy takes on **extra risk**?

## 4 Out‑of‑Sample Forecasting

_This section utilizes data in `gmo_data.xlsx`._ Focus on using **both** $DP$ and $EP$ as signals in (1). Compute **out‑of‑sample** ($OOS$) statistics:

**Procedure (rolling OOS):**
- Start at $t=60$.
- Estimate (1) using data **through** time $t$.
- Using the estimated parameters and $x_t$, compute the forecast for $t+1$:
  
  $$
  \hat r^{SPY}_{t+1} \;=\; \hat \alpha^{SPY,X}_t \;+\; \big(\hat \beta^{SPY,X}_t\big)^\prime x_t
  $$

- Forecast error: $e^{forecast}_{t+1} = r^{SPY}_{t+1} - \hat r^{SPY}_{t+1}$.
- Move to $t=61$ and iterate.

Also compute the **null** forecast and errors:

$$
\bar r^{SPY}_{t+1} = \frac{1}{t}\sum_{i=1}^t r^{SPY}_i, \qquad
e^{null}_{t+1} = r^{SPY}_{t+1} - \bar r^{SPY}_{t+1}.
$$

1. **Report the out‑of‑sample $R^2$**

$$
R^2_{OOS} \;\equiv\; 1 - \frac{\sum_{i=61}^T \big(e^{forecast}_i\big)^2}{\sum_{i=61}^T \big(e^{null}_i\big)^2}
$$

Did this forecasting strategy produce a positive $R^2_{OOS}$?

2. **Redo 3.2 with OOS forecasts.** How does the OOS strategy compare to the in‑sample version of 3.2?

3. **Redo 3.3 with OOS forecasts.** Is the point‑in‑time version of the strategy **riskier**?

## 5 EXTRA: ML Forecasts

1. **CART.** Re‑do Section 3 using **CART** (e.g., `RandomForestRegressor` from `sklearn.ensemble`). If you want to visualize, try `sklearn.tree`.
2. **CART, OOS.** Compute out‑of‑sample stats as in Section 4.
3. **Neural Network.** Re‑do Section 3 using a **neural network** (e.g., `MLPRegressor` from `sklearn.neural_network`).
4. **NN & CART, OOS.** Compute out‑of‑sample stats as in Section 4.